In [ ]:
using Revise
using QuantumCollocation
using NamedTrajectories
using LinearAlgebra
using SparseArrays
using Random; Random.seed!(123);

# Transmon System

Let's build a system describing mulitlevel transmon with Hamiltonian

$$
H(t) = -\frac{\delta}{2} a^\dagger a^\dagger a a + u_1(t) (a + a^\dagger) + u_2(t) i (a - a^\dagger)
$$

This system is implemented via the `TransmonSystem` constructor in `quantum_system_templates.jl`.

In [ ]:
sys = TransmonSystem(levels=4)

op = EmbeddedOperator(:X, sys)

T = 100
dt = 0.1

prob = UnitarySmoothPulseProblem(sys, op, T, dt; 
    a_bound=0.1
)

solve!(prob; max_iter=100)

println("fidelity = ", unitary_fidelity(prob, subspace=op.subspace_indices)^2)

In [ ]:
plot_unitary_populations(prob.trajectory; unitary_columns=1:2, res=(1000, 700))

# Leakage Suppression

We can enforce leakage suppression by adding an objective term of the form

$$
J_{\text{leakage}}(x) = \sum_{i,l} |U^{R}_{i,l}| + |U^{I}_{i,l}|
$$

This is $L_1$ norm regularizer on the real and imaginary parts of the $U_{i,l}$ unitary components, these are those components that transfer population out of the subspace into the leakage space.

In [ ]:
sys = TransmonSystem(levels=4)

op = EmbeddedOperator(:X, sys)

T = 100
dt = 0.1

prob_leakage = UnitarySmoothPulseProblem(sys, op, T, dt; 
    leakage_suppression=true, 
    R_leakage=1.0e-1,
    a_bound=0.05,
    a_guess=prob.trajectory.a
)

solve!(prob_leakage; max_iter=100)

println("fidelity = ", unitary_fidelity(prob_leakage, subspace=op.subspace_indices)^2)

In [ ]:
plot_unitary_populations(prob_leakage.trajectory; unitary_columns=1:2, res=(1000, 700))

In [ ]:
plot_unitary_populations(prob.trajectory; unitary_columns=1:2, res=(1000, 700))